# Pattern 8: Backtracking

## Overview

Backtracking is a systematic way to iterate through all possible solutions by building candidates incrementally and abandoning candidates ("backtracking") as soon as it determines the candidate cannot lead to a valid solution.

**When to use:**
- Generate all combinations/permutations
- Find all solutions to constraint satisfaction problems
- Puzzles (Sudoku, N-Queens, word search)
- "Find all ways to..." problems

**Key Insight:** Think of it as exploring a decision tree. Make a choice, explore, then undo (backtrack) and try another choice.

**Pattern**: Choose → Explore → Unchoose (Backtrack)

---

## Backtracking Fundamentals

### Core Template

```python
def backtrack(state, choices):
    # Base case: valid solution found
    if is_solution(state):
        result.append(state.copy())
        return
    
    # Explore all choices
    for choice in choices:
        if is_valid(choice, state):
            # Make choice
            state.add(choice)
            
            # Explore
            backtrack(state, remaining_choices)
            
            # Unmake choice (backtrack)
            state.remove(choice)
```

### Key Components

1. **State**: Current partial solution
2. **Choices**: Available options at current state
3. **Constraints**: Rules that determine valid choices
4. **Goal**: Condition for complete solution
5. **Pruning**: Early termination of invalid branches

### Decision Tree Metaphor

- **Root**: Starting state (empty solution)
- **Nodes**: Partial solutions
- **Edges**: Choices/decisions
- **Leaves**: Complete solutions or dead ends
- **Backtracking**: Moving back up the tree

---

## Example 1: Generate All Subsets

**Problem**: Given a set of distinct integers, generate all possible subsets (power set).

**Key Insight**: For each element, we have 2 choices: include it or exclude it.

In [ ]:
def subsets(nums):
    """
    Generate all subsets using backtracking.
    Time: O(2^n), Space: O(n) for recursion stack
    """
    result = []
    
    def backtrack(start, current):
        # Base case: we have a valid subset
        result.append(current.copy())
        
        # Try adding each remaining element
        for i in range(start, len(nums)):
            # Choose
            current.append(nums[i])
            
            # Explore
            backtrack(i + 1, current)
            
            # Unchoose (backtrack)
            current.pop()
    
    backtrack(0, [])
    return result

# Example
nums = [1, 2, 3]
result = subsets(nums)
print(f"Input: {nums}")
print(f"All subsets:")
for subset in result:
    print(f"  {subset}")
print(f"Total: {len(result)} subsets (2^{len(nums)} = {2**len(nums)})")

### Visualization: Subsets Decision Tree

In [ ]:
def subsets_visual(nums):
    """Visual walkthrough of subset generation."""
    result = []
    
    def backtrack(start, current, depth=0):
        indent = "  " * depth
        
        # Current state is a valid subset
        print(f"{indent}State: {current} → Add to result")
        result.append(current.copy())
        
        if start >= len(nums):
            print(f"{indent}(Leaf node)")
            return
        
        # Try adding each remaining element
        for i in range(start, len(nums)):
            print(f"{indent}Try adding {nums[i]}")
            
            # Choose
            current.append(nums[i])
            
            # Explore
            backtrack(i + 1, current, depth + 1)
            
            # Unchoose
            current.pop()
            print(f"{indent}Backtrack from {nums[i]}")
    
    print(f"Generating subsets for {nums}\n")
    backtrack(0, [])
    print(f"\nAll subsets: {result}")
    return result

subsets_visual([1, 2, 3])

---

## Example 2: Generate All Permutations

**Problem**: Generate all possible permutations of a list of distinct integers.

**Key Insight**: At each position, try all available (unused) elements.

In [ ]:
def permute(nums):
    """
    Generate all permutations using backtracking.
    Time: O(n! * n), Space: O(n)
    """
    result = []
    
    def backtrack(current, remaining):
        # Base case: no more elements to add
        if not remaining:
            result.append(current.copy())
            return
        
        # Try each remaining element
        for i in range(len(remaining)):
            # Choose
            current.append(remaining[i])
            
            # Explore with remaining elements (excluding current)
            backtrack(current, remaining[:i] + remaining[i+1:])
            
            # Unchoose
            current.pop()
    
    backtrack([], nums)
    return result

# Example
nums = [1, 2, 3]
result = permute(nums)
print(f"Input: {nums}")
print(f"All permutations:")
for perm in result:
    print(f"  {perm}")
print(f"Total: {len(result)} permutations (3! = 6)")

In [ ]:
# Alternative: Using used set instead of slicing
def permute_optimized(nums):
    """
    More efficient: use set to track used elements.
    Time: O(n! * n), Space: O(n)
    """
    result = []
    
    def backtrack(current, used):
        if len(current) == len(nums):
            result.append(current.copy())
            return
        
        for i, num in enumerate(nums):
            if i not in used:
                # Choose
                current.append(num)
                used.add(i)
                
                # Explore
                backtrack(current, used)
                
                # Unchoose
                current.pop()
                used.remove(i)
    
    backtrack([], set())
    return result

print("\nOptimized version:")
result = permute_optimized([1, 2, 3])
for perm in result:
    print(f"  {perm}")

### Visualization: Permutations Decision Tree

In [ ]:
def permute_visual(nums):
    """Visual walkthrough of permutation generation."""
    result = []
    
    def backtrack(current, remaining, depth=0):
        indent = "  " * depth
        
        # Base case
        if not remaining:
            print(f"{indent}✓ Complete permutation: {current}")
            result.append(current.copy())
            return
        
        print(f"{indent}Current: {current}, Remaining: {remaining}")
        
        for i, num in enumerate(remaining):
            print(f"{indent}Try adding {num}")
            
            # Choose
            current.append(num)
            new_remaining = remaining[:i] + remaining[i+1:]
            
            # Explore
            backtrack(current, new_remaining, depth + 1)
            
            # Unchoose
            current.pop()
            print(f"{indent}Backtrack from {num}")
    
    print(f"Generating permutations for {nums}\n")
    backtrack([], nums)
    print(f"\nAll permutations: {result}")
    return result

permute_visual([1, 2, 3])

---

## Example 3: Combination Sum

**Problem**: Find all unique combinations that sum to target. Can reuse elements.

**Key Insight**: For each element, decide: include it (and can reuse) or skip it.

In [ ]:
def combination_sum(candidates, target):
    """
    Find all combinations that sum to target.
    Can reuse elements unlimited times.
    Time: O(n^(target/min)), Space: O(target/min)
    """
    result = []
    
    def backtrack(start, current, remaining):
        # Base cases
        if remaining == 0:
            result.append(current.copy())
            return
        
        if remaining < 0:
            return  # Pruning: exceeded target
        
        # Try each candidate starting from 'start'
        for i in range(start, len(candidates)):
            # Choose
            current.append(candidates[i])
            
            # Explore (can reuse same element, so pass i not i+1)
            backtrack(i, current, remaining - candidates[i])
            
            # Unchoose
            current.pop()
    
    backtrack(0, [], target)
    return result

# Example
candidates = [2, 3, 6, 7]
target = 7
result = combination_sum(candidates, target)

print(f"Candidates: {candidates}")
print(f"Target: {target}")
print(f"Combinations:")
for combo in result:
    print(f"  {combo} (sum = {sum(combo)})")

### Visualization: Combination Sum with Pruning

In [ ]:
def combination_sum_visual(candidates, target):
    """Visual walkthrough showing pruning."""
    result = []
    
    def backtrack(start, current, remaining, depth=0):
        indent = "  " * depth
        
        # Base cases
        if remaining == 0:
            print(f"{indent}✓ Found solution: {current} (sum = {target})")
            result.append(current.copy())
            return
        
        if remaining < 0:
            print(f"{indent}✗ Pruned: {current} exceeds target")
            return
        
        print(f"{indent}Current: {current}, Remaining: {remaining}")
        
        for i in range(start, len(candidates)):
            num = candidates[i]
            print(f"{indent}Try adding {num}")
            
            # Choose
            current.append(num)
            
            # Explore
            backtrack(i, current, remaining - num, depth + 1)
            
            # Unchoose
            current.pop()
            print(f"{indent}Backtrack from {num}")
    
    print(f"Finding combinations for target {target} using {candidates}\n")
    backtrack(0, [], target)
    print(f"\nAll combinations: {result}")
    return result

combination_sum_visual([2, 3, 6, 7], 7)

---

## Example 4: N-Queens

**Problem**: Place N queens on N×N chessboard so no two queens attack each other.

**Constraints**:
- One queen per row
- One queen per column
- One queen per diagonal

**Key Insight**: Place queens row by row, checking constraints at each step.

In [ ]:
def solve_n_queens(n):
    """
    Solve N-Queens problem using backtracking.
    Time: O(n!), Space: O(n)
    """
    result = []
    board = [['.'] * n for _ in range(n)]
    
    def is_valid(row, col):
        # Check column
        for i in range(row):
            if board[i][col] == 'Q':
                return False
        
        # Check diagonal (top-left)
        i, j = row - 1, col - 1
        while i >= 0 and j >= 0:
            if board[i][j] == 'Q':
                return False
            i -= 1
            j -= 1
        
        # Check diagonal (top-right)
        i, j = row - 1, col + 1
        while i >= 0 and j < n:
            if board[i][j] == 'Q':
                return False
            i -= 1
            j += 1
        
        return True
    
    def backtrack(row):
        # Base case: all queens placed
        if row == n:
            result.append([''.join(row) for row in board])
            return
        
        # Try placing queen in each column
        for col in range(n):
            if is_valid(row, col):
                # Choose
                board[row][col] = 'Q'
                
                # Explore
                backtrack(row + 1)
                
                # Unchoose
                board[row][col] = '.'
    
    backtrack(0)
    return result

# Example
solutions = solve_n_queens(4)
print(f"4-Queens: {len(solutions)} solutions\n")

for i, solution in enumerate(solutions, 1):
    print(f"Solution {i}:")
    for row in solution:
        print(f"  {row}")
    print()

In [ ]:
# Optimized N-Queens using sets for O(1) constraint checking
def solve_n_queens_optimized(n):
    """
    Optimized with sets to track attacked positions.
    Time: O(n!), Space: O(n)
    """
    result = []
    board = [['.'] * n for _ in range(n)]
    
    # Track attacked columns and diagonals
    cols = set()
    diag1 = set()  # row - col
    diag2 = set()  # row + col
    
    def backtrack(row):
        if row == n:
            result.append([''.join(row) for row in board])
            return
        
        for col in range(n):
            # Check if position is attacked
            if col in cols or (row - col) in diag1 or (row + col) in diag2:
                continue
            
            # Choose
            board[row][col] = 'Q'
            cols.add(col)
            diag1.add(row - col)
            diag2.add(row + col)
            
            # Explore
            backtrack(row + 1)
            
            # Unchoose
            board[row][col] = '.'
            cols.remove(col)
            diag1.remove(row - col)
            diag2.remove(row + col)
    
    backtrack(0)
    return result

print("Optimized version:")
solutions = solve_n_queens_optimized(8)
print(f"8-Queens: {len(solutions)} solutions")

---

## Example 5: Word Search

**Problem**: Find if a word exists in a 2D board of characters.

**Approach**: DFS with backtracking, marking visited cells.

In [ ]:
def word_search(board, word):
    """
    Find if word exists in board using backtracking.
    Time: O(m*n*4^L) where L = word length
    Space: O(L) for recursion stack
    """
    if not board or not word:
        return False
    
    m, n = len(board), len(board[0])
    
    def backtrack(row, col, index):
        # Base case: found entire word
        if index == len(word):
            return True
        
        # Check bounds
        if (row < 0 or row >= m or col < 0 or col >= n or
            board[row][col] != word[index]):
            return False
        
        # Mark as visited
        temp = board[row][col]
        board[row][col] = '#'
        
        # Explore 4 directions
        found = (backtrack(row + 1, col, index + 1) or
                 backtrack(row - 1, col, index + 1) or
                 backtrack(row, col + 1, index + 1) or
                 backtrack(row, col - 1, index + 1))
        
        # Unmark (backtrack)
        board[row][col] = temp
        
        return found
    
    # Try starting from each cell
    for i in range(m):
        for j in range(n):
            if backtrack(i, j, 0):
                return True
    
    return False

# Example
board = [
    ['A', 'B', 'C', 'E'],
    ['S', 'F', 'C', 'S'],
    ['A', 'D', 'E', 'E']
]

print("Board:")
for row in board:
    print('  ', row)

test_words = ['ABCCED', 'SEE', 'ABCB']
print("\nWord Search:")
for word in test_words:
    found = word_search([row[:] for row in board], word)  # Copy board
    print(f"  '{word}': {found}")

---

## Common Backtracking Patterns

### 1. Basic Backtracking Template
```python
def backtrack(state):
    if is_solution(state):
        result.append(state.copy())
        return
    
    for choice in get_choices(state):
        if is_valid(choice, state):
            # Choose
            make_choice(state, choice)
            
            # Explore
            backtrack(state)
            
            # Unchoose
            undo_choice(state, choice)
```

### 2. Subsets/Combinations Template
```python
def backtrack(start, current):
    result.append(current.copy())
    
    for i in range(start, len(nums)):
        current.append(nums[i])
        backtrack(i + 1, current)  # i+1 to avoid duplicates
        current.pop()
```

### 3. Permutations Template
```python
def backtrack(current, used):
    if len(current) == len(nums):
        result.append(current.copy())
        return
    
    for i, num in enumerate(nums):
        if i not in used:
            current.append(num)
            used.add(i)
            backtrack(current, used)
            current.pop()
            used.remove(i)
```

### 4. Grid/Board Backtracking
```python
def backtrack(row, col):
    if is_goal(row, col):
        return True
    
    if not is_valid(row, col):
        return False
    
    # Mark as visited
    temp = board[row][col]
    board[row][col] = VISITED
    
    # Try all directions
    for dr, dc in [(0,1), (0,-1), (1,0), (-1,0)]:
        if backtrack(row+dr, col+dc):
            return True
    
    # Unmark (backtrack)
    board[row][col] = temp
    return False
```

### 5. With Pruning
```python
def backtrack(state, remaining):
    if remaining < 0:  # Pruning condition
        return
    
    if remaining == 0:
        result.append(state.copy())
        return
    
    for choice in choices:
        state.append(choice)
        backtrack(state, remaining - cost(choice))
        state.pop()
```

---

## Optimization: Pruning

**Pruning** = Early termination of branches that cannot lead to valid solutions.

### Common Pruning Strategies:

1. **Constraint Checking**: Skip invalid choices early
2. **Bound Checking**: Prune if exceeding limits
3. **Duplicate Avoidance**: Sort and skip duplicates
4. **Early Success**: Return immediately on first solution

In [ ]:
# Example: Pruning with sorting
def combination_sum_pruned(candidates, target):
    """
    Optimized with sorting and pruning.
    """
    result = []
    candidates.sort()  # Sort to enable pruning
    
    def backtrack(start, current, remaining):
        if remaining == 0:
            result.append(current.copy())
            return
        
        for i in range(start, len(candidates)):
            # Pruning: if current number > remaining, all later numbers will too
            if candidates[i] > remaining:
                break  # Stop early!
            
            current.append(candidates[i])
            backtrack(i, current, remaining - candidates[i])
            current.pop()
    
    backtrack(0, [], target)
    return result

result = combination_sum_pruned([2, 3, 6, 7], 7)
print("With pruning:", result)

---

## Practice Problems

### Easy
1. Subsets - Generate all subsets
2. Permutations - Generate all permutations
3. Letter Combinations of Phone Number - Map digits to letters

### Medium
4. Combination Sum - Find combinations that sum to target
5. Combination Sum II - No duplicate combinations
6. Permutations II - Permutations with duplicates
7. Subsets II - Subsets with duplicates
8. Palindrome Partitioning - Partition into palindromes
9. Word Search - Find word in grid
10. Generate Parentheses - Valid parentheses combinations
11. Restore IP Addresses - Generate valid IPs

### Hard
12. N-Queens - Place N queens on board
13. Sudoku Solver - Solve Sudoku puzzle
14. Word Search II - Find multiple words
15. Expression Add Operators - Insert operators to reach target

## Key Takeaways

- Backtracking explores decision tree systematically
- **Pattern**: Choose → Explore → Unchoose
- Always **backtrack** (undo choice) after exploring
- Use **pruning** to improve efficiency
- Common for: permutations, combinations, constraint problems
- Think recursively: solve smaller subproblem
- Time complexity often exponential: O(2^n), O(n!)

## Backtracking vs Other Approaches

| Technique | Use Case | Complexity |
|-----------|----------|------------|
| **Backtracking** | Generate all solutions, constraint problems | Exponential |
| **Dynamic Programming** | Optimization, count solutions | Polynomial |
| **Greedy** | Optimization with greedy choice | Polynomial |
| **DFS/BFS** | Graph traversal, shortest paths | O(V+E) |

## When to Use Backtracking

✅ **Use when:**
- Generate all combinations/permutations
- Find all valid solutions
- Constraint satisfaction problems (Sudoku, N-Queens)
- "Find all ways to..." problems
- Decision tree exploration

❌ **Don't use when:**
- Only need one solution (use greedy/DP)
- Overlapping subproblems (use DP)
- Large search space without pruning
- Simple iteration suffices

## Decision Tree vs Backtracking

| Aspect | Decision Tree | Backtracking |
|--------|---------------|---------------|
| **Concept** | Visual representation | Implementation technique |
| **Nodes** | States/choices | Function calls |
| **Edges** | Decisions | Recursive calls |
| **Traversal** | DFS | Recursion with undo |

---

**Congratulations!** You've completed all 8 core algorithm patterns.